In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging

In [2]:
# Set project root
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
print(project_root)

/Users/gnaneshreddysaireddy/TasteDrift-MovieLens


In [6]:
import os

os.listdir(project_root/"data/processed/")


['user_clusters.csv', 'ratings_with_time_split.zip', 'user_taste_vectors.csv']

In [7]:
# Configure logging
log_dir = project_root / "logs"
log_dir.mkdir(exist_ok=True)
log_file = log_dir / "feature_engineering.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Load processed data from Notebook 1
# data_path = project_root / "data/processed/ratings_with_time_split.zip"
# df = pd.read_csv(data_path, parse_dates=['datetime'])
import zipfile
import pandas as pd

zip_path = project_root/ "data/processed/ratings_with_time_split.zip"

with zipfile.ZipFile(zip_path, "r") as z:
    with z.open("ratings_with_time_split.csv") as f:
        df = pd.read_csv(f)

print(f"✅ Loaded from ZIP: {df.shape[0]:,} rows")

logger.info(f"Loaded processed data: {df.shape[0]} rows")
# Genre vector creation (user x genre matrix)
df['genres'] = df['genres'].apply(lambda g: g.split('|') if isinstance(g, str) else g)
df_exploded = df.explode('genres')

# Time bucket assignment
df_exploded['bucket'] = df_exploded['time_split']

grouped = df_exploded.groupby(['user_id', 'bucket', 'genres']).size().unstack(fill_value=0)
logger.info(f"Created user-genre matrix: {grouped.shape[0]} users x {grouped.shape[1]} genres")

# Save output for modeling
output_path = project_root / "data/processed/user_taste_vectors.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
grouped.to_csv(output_path)
logger.info(f"Saved user taste vectors to: {output_path}")

2025-07-31 15:32:21,694 - INFO - Loaded processed data: 1000209 rows


✅ Loaded from ZIP: 1,000,209 rows


2025-07-31 15:32:23,590 - INFO - Created user-genre matrix: 12080 users x 301 genres
2025-07-31 15:32:24,102 - INFO - Saved user taste vectors to: /Users/gnaneshreddysaireddy/TasteDrift-MovieLens/data/processed/user_taste_vectors.csv
